# Active Learning: Q1 (b): iii

In [0]:
'''
Q1(b)iii
What is the size (number of points) of the version space? Order points to label
in such a way that the version space gets reduced by maximum with each point
chosen to be labelled.
'''

'\nQ1(b)iii\nWhat is the size (number of points) of the version space? Order points to label\nin such a way that the version space gets reduced by maximum with each point\nchosen to be labelled.\n'



---

In this section, we shall be calculating the size of the Version space.
Moreover, we shall be placing the unlablled points in an order through which labellling the first points in this order results in maximum reduction of version spae.

Measure of disagreement used by the committee: **Vote Entropy**


---



In [1]:
import numpy as np 
import pandas as pd 
import math
from statistics import mean 
from sklearn.preprocessing import StandardScaler  
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split 
from sklearn import datasets
from scipy.stats import entropy
from time import time
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn import metrics

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# split dataset into test set, train set and unlabel pool 

def split(dataset, train_size, test_size): 
	x = dataset[:, :-1] 
	y = dataset[:, -1] 
	x_train, x_pool, y_train, y_pool = train_test_split( 
		x, y, train_size = train_size) 
	unlabel, x_test, label, y_test = train_test_split( 
		x_pool, y_pool, test_size = test_size) 
	return x_train, y_train, x_test, y_test, unlabel, label

In [0]:
# Function defining both the QBC sampling approaches

def qbc_sampling(name, unlabel, percent_of_samples, dataset, x_train, y_train):

    # We are using BAGGING TECHNIQUE for forming our Committee

    #################################################################################
    clf1 = BaggingClassifier(base_estimator=SVC(), n_estimators=10, random_state=0)    
    clf1.fit(x_train, y_train)
    clf1_pred = clf1.predict(unlabel)

    clf2 = BaggingClassifier(base_estimator=SVC(), n_estimators=20, random_state=0)    
    clf2.fit(x_train, y_train)
    clf2_pred = clf2.predict(unlabel)

    clf3 = BaggingClassifier(base_estimator=SVC(), n_estimators=30, random_state=0)    
    clf3.fit(x_train, y_train)
    clf3_pred = clf3.predict(unlabel)

    clf4 = BaggingClassifier(base_estimator=SVC(), n_estimators=40, random_state=0)    
    clf4.fit(x_train, y_train)
    clf4_pred = clf4.predict(unlabel)

    clf5 = BaggingClassifier(base_estimator=SVC(), n_estimators=50, random_state=0)    
    clf5.fit(x_train, y_train)
    clf5_pred = clf5.predict(unlabel)

    # #Classifier 1: Logistic Regression
    # clf1 = LogisticRegression() 
    # x_train, y_train, x_test, y_test, unlabel, label = split( dataset, 0.10, 0.20)
    # clf1.fit(x_train, y_train)
    # clf1_pred = clf1.predict(unlabel)

    # #Classifier 2: RandomForestClassifier
    # clf2 = RandomForestClassifier(n_estimators=10) 
    # x_train, y_train, x_test, y_test, unlabel, label = split( dataset, 0.10, 0.20)
    # clf2.fit(x_train, y_train)
    # clf2_pred = clf2.predict(unlabel)

    # #Classifier 3: AdaBoostClassifier
    # clf3 = AdaBoostClassifier(n_estimators=100) 
    # x_train, y_train, x_test, y_test, unlabel, label = split( dataset, 0.10, 0.20)
    # clf3.fit(x_train, y_train)
    # clf3_pred = clf3.predict(unlabel)

    # #Classifier 4: BaggingClassifier
    # clf4 = BaggingClassifier(base_estimator=SVC(), n_estimators=20, random_state=0)    
    # x_train, y_train, x_test, y_test, unlabel, label = split( dataset, 0.10, 0.20)
    # clf4.fit(x_train, y_train)
    # clf4_pred = clf4.predict(unlabel)

    # #Classifier 5: GradientBoostingClassifier
    # clf5 = GradientBoostingClassifier(random_state=0)
    # x_train, y_train, x_test, y_test, unlabel, label = split( dataset, 0.10, 0.20)
    # clf5.fit(x_train, y_train)
    # clf5_pred = clf5.predict(unlabel)



    #################################################################################

    if name == 'vote_entropy':
        
        votes = []
        votes_prob = []
        for i in range(clf4_pred.shape[0]):
            temp = []
            temp.append(clf1_pred[i])
            temp.append(clf2_pred[i])
            temp.append(clf3_pred[i])
            temp.append(clf4_pred[i])
            temp.append(clf5_pred[i])

            votes.append(temp)

            temp_p = []
            for m in range(10):
                temp_p.append(votes[i].count(m) / len(votes[i]))

            votes_prob.append(temp_p)

        votes = []
        votes_prob = []
        for i in range(clf4_pred.shape[0]):
            temp = []
            temp.append(clf1_pred[i])
            temp.append(clf2_pred[i])
            temp.append(clf3_pred[i])
            temp.append(clf4_pred[i])
            temp.append(clf5_pred[i])

            votes.append(temp)

            temp_p = []
            for m in range(10):
                temp_p.append(votes[i].count(m) / len(votes[i]))

            votes_prob.append(temp_p)

        from scipy.stats import entropy
        entr = entropy(np.array(votes_prob).T)
        sorted_idx = np.argsort(entr)
        #print('Probability distribution of Top 5 Selection through ENTROPY: \n')
        #for i in range(1,6):
            #print("Selection #" + str(i) + ': ' + str(votes[sorted_idx[len(entr) - i]]) + '\n')
        uncrt_pt_ind = [] 
        for i in range(math.floor(percent_of_samples * 0.01 * dataset.shape[0])):
                uncrt_pt_ind.append(sorted_idx[-i-1])

        

        return uncrt_pt_ind, entr

    if name == 'kl_divergence':
        proba_votes = []
        proba_votes.append(clf1.predict_proba(unlabel))
        proba_votes.append(clf2.predict_proba(unlabel))
        proba_votes.append(clf3.predict_proba(unlabel))
        proba_votes.append(clf4.predict_proba(unlabel))
        proba_votes.append(clf5.predict_proba(unlabel))

        consensus_proba = (proba_votes[0] + proba_votes[1] + proba_votes[2] + proba_votes[3] + proba_votes[4]) / len(proba_votes)

        from scipy.stats import entropy
        learner_KL_div = []
        for i in range(proba_votes[1].shape[0]):
            temp = []
            
            for j in range(len(proba_votes)):
                temp.append(entropy(proba_votes[j][i], qk= consensus_proba[i]))

            learner_KL_div.append(temp)

        kl_max = np.array(learner_KL_div).max(axis=1)
        #print('Size of kl_max= '+str(kl_max.shape))
        #print('Size of unlabel= ' + str(unlabel.shape))
        sorted_idx = np.argsort(kl_max)
        #print('Probability distribution of Top 5 Selection through LEAST CONFIDENCE: \n')
        #for i in range(1,6):
            #print("Selection #" + str(i) + ': ' + str(proba[sorted_idx[i-1]]) + '\n')

        uncrt_pt_ind = [] 
        for i in range(math.floor(percent_of_samples * 0.01 * dataset.shape[0])):
            uncrt_pt_ind.append(sorted_idx[-i-1])

        return uncrt_pt_ind, learner_KL_div


In [0]:
# Funtion for calculating the version space

def calculate_version_space(entropy, threshold, ulabel):

    version_space_idx = []
    idx = 0
    for i in (entropy):
        idx = idx+1
        if i >= threshold:
            version_space_idx.append(idx-1)

    print('Size of Version Space: ' + str(len(version_space_idx)) + '  ( ' + str(len(version_space_idx) * 100 / ulabel.shape[0]) + ' % of points)' )
    
    return version_space_idx


In [0]:
# Preparing Data:

# Loading dataset
X_digits, y_digits = datasets.load_digits(return_X_y=True)
stack = np.vstack((X_digits.T, y_digits.T))
dataset = stack.T

# feature scalling 
sc = StandardScaler() 
dataset[:, :-1] = sc.fit_transform(dataset[:, :-1]) 

# Splitting Data
percent_of_samples = 10
train_size = 0.10
test_size = 0.20

x_train, unlabel, y_train, label = train_test_split(X_digits, y_digits, train_size = train_size)

In [0]:
# Placing the first 10% points to be labelled in order for maximum reduction in version space!

uncrt_pt_ind, entropy = qbc_sampling('vote_entropy', unlabel, percent_of_samples, dataset, x_train, y_train)

In [20]:
# CURRENT VERSION SPACE: Before using 10% of the data points with maximum disagreement
vs_idx_1 = calculate_version_space(entropy, 0.001, unlabel)

Size of Version Space: 13  ( 0.9034051424600417 % of points)


In [0]:
# Appending the newly labelled 10% data points into the training set
x_train = np.append(unlabel[uncrt_pt_ind, :], x_train, axis = 0) 
#proba_chosen_samples =  classifier1.predict_proba(unlabel[uncrt_pt_ind, :])
y_train = np.append(label[uncrt_pt_ind], y_train)

# Deleting the newly labelled data points from the unlabelled pool
unlabel = np.delete(unlabel, uncrt_pt_ind, axis = 0) 
label = np.delete(label, uncrt_pt_ind) 

In [9]:
# NEW VERSION SPACE: After using 10% of the data points with maximum disagreement
uncrt_pt_ind, entropy = qbc_sampling('vote_entropy', unlabel, percent_of_samples, dataset, x_train, y_train)
vs_idx_2 = calculate_version_space(entropy, 0.01, unlabel)


Size of Version Space: 13  ( 0.9034051424600417 % of points)


In [10]:
print('Initial size of the Version Space: ' + str(len(vs_idx_1)))
print('Final size of the Version Space: ' + str(len(vs_idx_2)))
print()

print('Reduction in Version Space (by labelling 10% of the most disagreed points) = ' + str((len(vs_idx_1) - len(vs_idx_2)) * 100 / len(vs_idx_1)) + ' %')

Initial size of the Version Space: 33
Final size of the Version Space: 13

Reduction in Version Space (by labelling 10% of the most disagreed points) = 60.60606060606061 %


END OF Q1(b)iii